In [72]:
# Clone the repo
!git clone https://github.com/ljeong072/TCSS456

Cloning into 'TCSS456'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), done.


In [73]:
import os

# Set directory into cloned repo and open the files to check.
os.chdir("TCSS456")  
os.listdir()

['README.md', '.git']

In [84]:
# See available branches
!git branch -a

* Dev
  main
  remotes/origin/Dev
  remotes/origin/HEAD -> origin/main
  remotes/origin/mahri
  remotes/origin/main


In [86]:
# Checkout Dev branch
!git checkout Dev

Already on 'Dev'
Your branch is up to date with 'origin/Dev'.


In [88]:
# Status of branch
!git status

On branch Dev
Your branch is up to date with 'origin/Dev'.

nothing to commit, working tree clean


In [79]:
# Push to Github (Change the message and check that this is the correct branch
!git add .
!git commit -m "First commit"
!git push
!git push origin Dev

On branch Dev
Your branch is up to date with 'origin/Dev'.

nothing to commit, working tree clean
Everything up-to-date
Everything up-to-date


In [4]:
# 1.1 Prerequisites
!pip install transformers datasets

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset

# Login using e.g. `huggingface-cli login` to access this dataset

dataset = load_dataset("thomasat/diet-planning")
# Change to this one dataset
#from datasets import load_dataset

# Split the train split into 80%/20% test
split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

# Access the new splits
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

#dataset = load_dataset("cnn_dailymail", "3.0.0")

# Load the dataset
#dataset["train"] = dataset["train"].select(range(100))
#dataset["validation"] = dataset["validation"].select(range(10))

# Test split will not be used
#dataset["test"] = dataset["test"].select(range(0))


#print(dataset)
#print(dataset['train'][10])
#print(dataset['validation'][5])

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Prompt', 'Formal specification', 'Max per-meal sodium (mg)', 'Max daily saturated fat (g)', 'Max daily calories (kcal)', 'Min daily fiber (g)', 'Min daily servings of vegetables', 'Min daily servings of fruit', 'Max daily percentage carbohydrates', 'Min daily percentage carbohydrates', 'Max daily percentage fat', 'Min daily percentage fat', 'Max daily percentage protein', 'Min daily percentage protein', 'Max daily servings of whole grains', 'Min daily servings of whole grains', 'Max servings of sweets per week', 'Max servings of red meat per week', 'Min servings of nuts, seeds, legumes per week', 'Max servings of low-fat dairy per week', 'Min servings of low-fat dairy per week', 'Max servings of fish per week', 'Min servings of fish per week', 'Dietary preference', 'Flavor preference', 'Cooking preference', 'Cuisine preference'],
        num_rows: 10000
    })
})


In [53]:
# 1.3 Step 2: Tokenize the Text
# Note: We have used distilbert-base-uncased tokenizer in Tutorial_1

from transformers import GPT2Tokenizer, GPT2Model

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the model (GPT-2 Small)
model = GPT2Model.from_pretrained('gpt2')

def tokenize_function(example):
  return tokenizer(example["Prompt"])

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["Prompt"])
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['Formal specification', 'Max per-meal sodium (mg)', 'Max daily saturated fat (g)', 'Max daily calories (kcal)', 'Min daily fiber (g)', 'Min daily servings of vegetables', 'Min daily servings of fruit', 'Max daily percentage carbohydrates', 'Min daily percentage carbohydrates', 'Max daily percentage fat', 'Min daily percentage fat', 'Max daily percentage protein', 'Min daily percentage protein', 'Max daily servings of whole grains', 'Min daily servings of whole grains', 'Max servings of sweets per week', 'Max servings of red meat per week', 'Min servings of nuts, seeds, legumes per week', 'Max servings of low-fat dairy per week', 'Min servings of low-fat dairy per week', 'Max servings of fish per week', 'Min servings of fish per week', 'Dietary preference', 'Flavor preference', 'Cooking preference', 'Cuisine preference', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [83]:
#1.4 Step 3: Group Tokens for Language Modeling
block_size = 236

def group_texts(examples):
    concatenated = sum(examples['input_ids'], [])
    concatenated_attention_mask = sum(examples['attention_mask'], [])
    total_length = (len(concatenated) // block_size) * block_size

    result = {
        'input_ids': [concatenated[i:i + block_size] for i in range(0, total_length, block_size)],
        'attention_mask': [concatenated_attention_mask[i:i + block_size] for i in range(0, total_length, block_size)]
    }

    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)
lm_dataset["train"][0]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

ArrowInvalid: Column 26 named input_ids expected length 1000 but got length 993

In [ ]:
# 1.5 Step 4: Load a Pretrained Model for Language Modeling

from transformers import AutoModelForCausalLM

model_name = "gpt2"

model = AutoModelForCausalLM.from_pretrained(model_name) # Note: we are using gpt2

In [ ]:
# Trains the model and selects a GPU if possible
# Some hyperparameters were changed as Google Colab's TPU and GPU
# had expired, so the CPU was utilized with parameters adjusted to
# allow it to train in a reasonable amount of time.

import transformers
from transformers import TrainingArguments, Trainer
import torch

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
  print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# 1. Leverage mixed precision training
training_args = TrainingArguments(
  output_dir="./lm_checkpoints",
  # 2. Aligned evaluation and saving strategy
    
  eval_strategy="steps", # Evaluate at specific steps
  eval_steps=100, # Evaluate every 100 steps

  save_strategy="steps", # Does not allow equality to "epoch"

  #save_steps=100,
  save_steps=1000, # Save every 1000 steps
  save_total_limit=2, # Keep only the 2 most recent checkpoints

  # 3. Increase learning rate and use warmup
  learning_rate=5e-5, # Higher learning rate
  warmup_ratio=0.1, # Warm up for first 10% of training

  # 4. Increase batch size
  per_device_train_batch_size=16, # Increase if your GPU has enough memory
  per_device_eval_batch_size=16,
  gradient_accumulation_steps=2, # Simulate larger batch sizes

  # 5. Enable fp16 training (mixed precision)
  fp16=True, # Enable mixed precision training

  # 6. Early stopping configuration
  load_best_model_at_end=True, # Load the best model when training ends
  metric_for_best_model="loss", # Use evaluation loss as the metric to track
  greater_is_better=False, # Lower loss is better

  # 7. Other optimizations
  weight_decay=0.01,
  logging_steps=50, # Less frequent logging
  report_to="none",

  # 8. Enable data parallelism if multiple GPUs are available
  dataloader_num_workers= 2, # Use multiple CPU cores for data loading
  # 9. Set number of epochs
  num_train_epochs=1, # Maintain the original epochs setting
)
# 9. Initialize the Trainer

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_dataset["train"],
  eval_dataset=lm_dataset["test"]
)
# 10. Optional: Use early stopping
early_stopping_callback = transformers.EarlyStoppingCallback(
  early_stopping_patience=3,
  early_stopping_threshold=0.01
)
trainer.add_callback(early_stopping_callback)

# Start training
trainer.train()

In [ ]:
from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
output1 = generator("Raging wildfires shook California as", max_length=40, num_return_sequences=1)
output2 = generator("Harry Potter is a book", max_length = 40, num_return_sequences=1)
output3 = generator("The Republic of Korea", max_length=40, num_return_sequences=1)

# 3 unique sentences for generation testing
print(output1)
print(output2)
print(output3)

In [ ]:
!pip install tf-keras

In [ ]:
%pip install transformers[torch]

In [ ]:
%pip install 'accelerate>=0.26.0'